In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
from os.path import join, expanduser, splitext, basename
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal
import rasterio as rio
import rioxarray as rxr
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from holoviews.streams import SingleTap
from scipy.stats import linregress



In [ ]:
FIG_WIDTH_PX = 1080
FIG_HEIGHT_PX = 720
FIG_WIDTH_IN = 16
FIG_HEIGHT_IN = 9
FIG_ALPHA = 0.7
BASEMAP = "EsriImagery"
SEABORN_STYLE = "whitegrid"
FILL_COLOR = "none"
LINE_COLOR = "red"
LINE_WIDTH = 3
WEBMAP_PROJECTION = "EPSG:4326"
sns.set_style(SEABORN_STYLE)


In [ ]:
data_dir = "HYRSENSE drought" 
data_dir

In [ ]:
ST_K = rxr.open_rasterio(ST_filename).squeeze('band', drop=True)
ST_C = ST_K - 273.15

ST_C.rio.to_raster(join(data_dir,"ST.tif"))

ST_vmin, ST_vmax = np.nanquantile(np.array(ST_C), [0.02, 0.98])
ST_title = "ECOSTRESS Surface Temperature (Celsius)"

ST_C.rio.reproject(WEBMAP_PROJECTION,nodata=np.nan).hvplot.image(
    geo=True,
    rasterize=True, 
    cmap="jet", 
    alpha=FIG_ALPHA, 
    tiles=BASEMAP,
    clim=(ST_vmin, ST_vmax),
    title=ST_title
)

In [ ]:
ST_filename = join(data_dir, 'ECOv002_L2T_LSTE_27352_012_13TDE_20230503T142012_0710_01_LST.tif') 
ST_filename


In [ ]:
def interpolate_hex(hex1, hex2, ratio):
    rgb1 = [int(hex1[i:i+2], 16) for i in (1, 3, 5)]
    rgb2 = [int(hex2[i:i+2], 16) for i in (1, 3, 5)]
    rgb = [int(rgb1[i] + (rgb2[i] - rgb1[i]) * ratio) for i in range(3)]
    
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

def create_gradient(colors, steps):
    gradient = []
    
    for i in range(len(colors) - 1):
        for j in range(steps):
            ratio = j / float(steps)
            gradient.append(interpolate_hex(colors[i], colors[i+1], ratio))
    
    gradient.append(colors[-1])
    
    return gradient

def plot_cmap(cmap):
    gradient = np.linspace(0, 1, 256)  # Gradient from 0 to 1
    gradient = np.vstack((gradient, gradient))  # Make 2D image

    # Display the colormap
    plt.figure(figsize=(6, 2))
    plt.imshow(gradient, aspect='auto', cmap=cmap)
    plt.axis('off')
    plt.show()

In [ ]:
ST_colors = [
    "#054fb9",
    "#0073e6",
    "#8babf1",
    "#cccccc",
    "#e1ad01",
    "#f57600",
    "#c44601"
]

ST_gradient = create_gradient(ST_colors, 100)
ST_cmap = LinearSegmentedColormap.from_list(name="ST", colors=ST_colors)
plot_cmap(ST_cmap)

In [ ]:
ST_vmin, ST_vmax = np.nanquantile(np.array(ST_C), [0.02, 0.98])
ST_title = "ECOSTRESS Surface Temperature (Celsius)"

ST_C.rio.reproject(WEBMAP_PROJECTION).hvplot.image(
    geo=True,
    rasterize=True, 
    cmap=ST_gradient, 
    alpha=FIG_ALPHA, 
    tiles=BASEMAP,
    clim=(ST_vmin, ST_vmax),
    title=ST_title
)

In [ ]:
ST_filename = join(data_dir, 'ECOv002_L2T_LSTE_29652_004_13TDE_20230928T210002_0711_01_LST.tif') 
ST_filename

In [ ]:
ST_K = rxr.open_rasterio(ST_filename).squeeze('band', drop=True)
ST_C = ST_K - 273.15

ST_C.rio.to_raster(join(data_dir,"ST.tif"))

ST_vmin, ST_vmax = np.nanquantile(np.array(ST_C), [0.02, 0.98])
ST_title = "ECOSTRESS Surface Temperature (Celsius)"

ST_C.rio.reproject(WEBMAP_PROJECTION,nodata=np.nan).hvplot.image(
    geo=True,
    rasterize=True, 
    cmap="jet", 
    alpha=FIG_ALPHA, 
    tiles=BASEMAP,
    clim=(ST_vmin, ST_vmax),
    title=ST_title
)

In [ ]:
ST_vmin, ST_vmax = np.nanquantile(np.array(ST_C), [0.02, 0.98])
ST_title = "ECOSTRESS Surface Temperature (Celsius)"

ST_C.rio.reproject(WEBMAP_PROJECTION).hvplot.image(
    geo=True,
    rasterize=True, 
    cmap=ST_gradient, 
    alpha=FIG_ALPHA, 
    tiles=BASEMAP,
    clim=(ST_vmin, ST_vmax),
    title=ST_title
)

In [ ]:
import geopandas as gp
import rasterio as rio
import rioxarray as rxr
import hvplot.xarray
import hvplot.pandas

In [ ]:
size_opts = dict(frame_height=405, frame_width=720, fontscale=2)
map_opts = dict(geo=True, crs='EPSG:4326', alpha=0.7, xlabel='Longitude',ylabel='Latitude')

In [ ]:
shape = gp.read_file("HYRSENSE drought/ROMO_BND_Boundary_py_NAD83_1_-2935172777854153534.geojson")
shape

In [ ]:
eco_fp = "HYRSENSE drought/ECOv002_L2T_LSTE_27352_012_13TDE_20230503T142012_0710_01_LST.tif"

In [ ]:
eco_lst_ds = rxr.open_rasterio(eco_fp).squeeze('band', drop=True)
eco_lst_ds

In [ ]:
eco_lst_ds.rio.reproject('EPSG:4326').hvplot.image(x='x',y='y',**size_opts, cmap='inferno',tiles='ESRI', xlabel='Longitude',ylabel='Latitude',title='ECOSTRESS LST (F)',)*shape.hvplot(color='#d95f02',alpha=0.5, )


In [ ]:
eco_cropped = eco_lst_ds.rio.clip(shape.geometry.values,shape.crs, all_touched=True)

In [ ]:
eco_lst_ds.rio.reproject('EPSG:4326').hvplot.image(x='x',y='y',**size_opts, cmap='inferno',tiles='ESRI', xlabel='Longitude',ylabel='Latitude',title='ECOSTRESS LST (K)', crs='EPSG:4326')


In [ ]:
eco_lst_ds = rxr.open_rasterio(eco_fp).squeeze('band', drop=True)
eco_lst_ds

In [ ]:
eco_lst_ds.rio.reproject('EPSG:4326').hvplot.image(x='x',y='y',**size_opts, cmap='inferno',tiles='ESRI', xlabel='Longitude',ylabel='Latitude',title='ECOSTRESS LST (K)', crs='EPSG:4326')


In [ ]:
eco_lst_ds_regrid = eco_lst_ds.rio.reproject_match
